#### Code to extract Method name form the python script

Reference:
Parsing Python Abstract Syntax Trees to Get Method Names
https://suhas.org/function-call-ast-python/ 

In [2]:
import ast

In [3]:
tree = ast.parse(open('sample.py').read())
ast.dump(tree)

"Module(body=[Import(names=[alias(name='foo', asname=None)]), Assign(targets=[Name(id='data', ctx=Store())], value=Call(func=Name(id='open', ctx=Load()), args=[Str(s='file')], keywords=[], starargs=None, kwargs=None)), Expr(value=Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='bar', ctx=Load()), args=[], keywords=[keyword(arg='arg', value=Name(id='data', ctx=Load()))], starargs=None, kwargs=None)), Expr(value=Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='bar', ctx=Load()), args=[], keywords=[keyword(arg='arg', value=Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='meow', ctx=Load()), args=[Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='z', ctx=Load()), args=[], keywords=[keyword(arg='arg', value=Name(id='data', ctx=Load()))], starargs=None, kwargs=None)], keywords=[], starargs=None, kwargs=None))], starargs=None, kwargs=None)), Expr(value=Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='woof', ctx=Load()), args=[Call(func=Att

In [4]:
tree

In [5]:
tree.body

we get a fair idea that every part of our python statements are internally represented as an AST by some object of the ast class.

    ast.Import for import foo.
    ast.Assign for data = open('file')
    ast.Expr each for the following function calls.


In [8]:
for statement in tree.body:
    print ast.dump(statement), '\n'

Import(names=[alias(name='foo', asname=None)]) 

Assign(targets=[Name(id='data', ctx=Store())], value=Call(func=Name(id='open', ctx=Load()), args=[Str(s='file')], keywords=[], starargs=None, kwargs=None)) 

Expr(value=Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='bar', ctx=Load()), args=[], keywords=[keyword(arg='arg', value=Name(id='data', ctx=Load()))], starargs=None, kwargs=None)) 

Expr(value=Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='bar', ctx=Load()), args=[], keywords=[keyword(arg='arg', value=Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='meow', ctx=Load()), args=[Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='z', ctx=Load()), args=[], keywords=[keyword(arg='arg', value=Name(id='data', ctx=Load()))], starargs=None, kwargs=None)], keywords=[], starargs=None, kwargs=None))], starargs=None, kwargs=None)) 

Expr(value=Call(func=Attribute(value=Name(id='foo', ctx=Load()), attr='woof', ctx=Load()), args=[Call(func=Attribute(val

The Call object is what we want to explore, which appears as the child object of Expr or Assign objects in our case.

from http://greentreesnakes.readthedocs.io/en/latest/nodes.html#Call

What we’ve learned so far:

    Every statement is represented as an object of one of the ast classes
    Function calls are ast.Call objects

What we need to be careful of:

    The recursive nature of the AST: nested calls like foo(bar()) are represented by an ast.Call object within an ast.Call object.


#### Traversing the AST

It is possible to do a depth-first traversal of the node by sub-classing ast.NodeVisitor and implementing a visit_* method; which can be any of the ast classes like visit_Call, visit_Assign, visit_Expr, etc.

It is possible to do a depth-first traversal of the node by sub-classing ast.NodeVisitor and implementing a visit_* method; which can be any of the ast classes like visit_Call, visit_Assign, visit_Expr, etc.

Let’s visit every function call.

In [12]:
func_calls=[]

for node in ast.walk(tree):
    if isinstance(node, ast.Call):
        api_name =''
        method_name =''
        for subnode in ast.walk(node):
            if isinstance(subnode, ast.Attribute):
                #print method name
                method_name = subnode.attr
                
                for n in ast.walk(subnode.value):
                    if isinstance(n, ast.Name):
                        api_name = n.id
        print api_name+'.'+method_name        

.
foo.bar
foo.z
foo.x
foo.x
foo.z
foo.z


In [22]:
func_calls=[]

for node in ast.walk(tree):
    if isinstance(node, ast.Call):
        visitor = ast.NodeVisitor()
        visitor.visit(node.func)
        print ast.dump(visitor)

TypeError: expected AST, got 'NodeVisitor'